In [2]:
import re
import pickle
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity


KeyboardInterrupt: 

In [ ]:

# Load NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


: 

In [ ]:

# Preprocess text (tokenization, stemming, lemmatization, and lowercasing)
def preprocess_text(text):
    text = re.sub(r'\W+', ' ', text)  # Remove non-alphanumeric characters
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenize text
    tokens = [stemmer.stem(word) for word in tokens]  # Apply stemming
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Apply lemmatization
    return ' '.join(tokens)  # Join tokens back into a single string


: 

In [ ]:

# Load model and embeddings from pickle file
with open('model/keelworks_model.pkl', 'rb') as f:
    model_data = pickle.load(f)

model = model_data['model']
faq_questions = model_data['faq_questions']
faq_embeddings = model_data['faq_embeddings']
faqs = model_data['faqs']


: 

In [ ]:
# Generate SBERT embeddings
def get_sbert_embedding(text):
    embedding = model.encode(text)
    return embedding

# Find the best matching answer
def get_best_answer(user_query, faqs, faq_embeddings, threshold=0.5):
    preprocessed_query = preprocess_text(user_query)
    query_embedding = get_sbert_embedding(preprocessed_query).reshape(1, -1)

    similarities = cosine_similarity(query_embedding, faq_embeddings)
    best_match_index = similarities.argmax()
    best_match_score = similarities[0, best_match_index]

    if best_match_score < threshold:
        return "Sorry, I don't have the answer. Please email to test@keelworks to get more info."
    return faqs[best_match_index]['answer']


: 

In [ ]:

# Command-Line Interface
def chatbot():
    print("Welcome to the KeelWorks Chatbot!")
    user_name = input("Please enter your name: ")
    print(f"Hello {user_name}, welcome to the KeelWorks bot. Ask me anything about KeelWorks.")

    while True:
        user_query = input("\nYou: ")
        if user_query.lower() in ['exit', 'quit', 'bye']:
            print(f"Goodbye, {user_name}!")
            break
        answer = get_best_answer(user_query, faqs, faq_embeddings)
        print(f"Bot: {answer}")

if __name__ == '__main__':
    chatbot()

: 

: 